In [1]:
#--------------Authors----------------
# t.acosta
# da.rubioh
# d.alvarezp

In [2]:
import nltk

In [3]:
nltk.download('stopwords')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Instalación de librerias
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport
from sklearn.naive_bayes import MultinomialNB
import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
import scikitplot as skplt
from sklearn.base import BaseEstimator, ClassifierMixin
import unicodedata
import re
from inflect import engine
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt


C:\Users\user\AppData\Local\Temp\ipykernel_16776\37452716.py:6: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [5]:
data=pd.read_excel('cat_345.xlsx')

In [6]:
train=data

In [7]:
train

Textos_espanol  sdg
0     Por ejemplo, el nÃºmero de consultas externas ...    3
1     En 2007, el gobierno central financió directam...    3
2     Claramente, hay muchos otros factores en juego...    3
3     Por ejemplo, el estado australiano de Victoria...    3
4     El consumo anual de alcohol se estima en 15,7 ...    3
...                                                 ...  ...
2995  Un caucus efectivo se basa en fuertes vínculos...    5
2996  Por el contrario, el porcentaje de hogares en ...    5
2997  El análisis utilizará una gama de medidas que ...    5
2998  La capacitación económica, el apoyo y, a veces...    5
2999  Esto ha sido reconocido por los comitÃ©s en la...    5

[3000 rows x 2 columns]

In [8]:
textos = train.copy()
textos['Conteo'] = [len(x) for x in textos['Textos_espanol']]
textos['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in textos['Textos_espanol']]
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['Textos_espanol']]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['Textos_espanol']]

# Se realiza un perfilamiento de los datos con la libre pandas profiling
ProfileReport(textos)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Preparación de datos

* Limpieza de los datos.
* Tokenización.
* Normalización.
* Vectorizacion

In [9]:
from langdetect import detect


In [10]:

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('spanish'):
            new_words.append(word)
    return new_words

def remove_specialCoders(words):
    new_words = []
    for word in words:
        if "ao" in word:
            new_word = re.sub(r'ao', 'ú', word)
            new_words.append(new_word)
        elif "a3" in word:
            new_word = re.sub(r'a3', 'ó', word)
            new_words.append(new_word)
        elif "a3" in word:
            new_word = re.sub(r'Ã¡', 'á', word)
            new_words.append(new_word)
        elif "a3" in word:
            new_word = re.sub(r'Ã©', 'é', word)
            new_words.append(new_word)
        elif "a3" in word:
            new_word = re.sub(r'Ã', 'í', word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def replace_numbers(words, p):
    """Replace all integer occurrences in a list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def preprocessing(words):
    p = engine()
    words = to_lowercase(words)
    words = replace_numbers(words, p)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    words = remove_specialCoders(words)
    return words


## **3.2 Tokenización**
La tokenización permite dividir frases u oraciones en palabras. Con el fin de desglozar las palabras correctamente para el posterior análisis. Pero primero, se realiza una corrección de las contracciones que pueden estar presentes en los textos. 

In [11]:
train['Textos_espanol'] = train['Textos_espanol'].apply(contractions.fix) #Aplica la corrección de las contracciones

In [12]:
train['words'] = train['Textos_espanol'].apply(word_tokenize).apply(preprocessing) #Aplica la eliminación del ruido
train.head()

Textos_espanol  sdg  \
0  Por ejemplo, el nÃºmero de consultas externas ...    3   
1  En 2007, el gobierno central financió directam...    3   
2  Claramente, hay muchos otros factores en juego...    3   
3  Por ejemplo, el estado australiano de Victoria...    3   
4  El consumo anual de alcohol se estima en 15,7 ...    3   

                                               words  
0  [ejemplo, número, consultas, externas, especia...  
1  [two thousand and seven, gobierno, central, fi...  
2  [claramente, factores, juego, particular, vari...  
3  [ejemplo, australiano, victoria, creo, agencia...  
4  [consumo, anual, alcohol, estima, 157, litros,...

In [13]:
train.head()

Textos_espanol  sdg  \
0  Por ejemplo, el nÃºmero de consultas externas ...    3   
1  En 2007, el gobierno central financió directam...    3   
2  Claramente, hay muchos otros factores en juego...    3   
3  Por ejemplo, el estado australiano de Victoria...    3   
4  El consumo anual de alcohol se estima en 15,7 ...    3   

                                               words  
0  [ejemplo, número, consultas, externas, especia...  
1  [two thousand and seven, gobierno, central, fi...  
2  [claramente, factores, juego, particular, vari...  
3  [ejemplo, australiano, victoria, creo, agencia...  
4  [consumo, anual, alcohol, estima, 157, litros,...

#### **3.3. Normalización**
En la normalización de los datos se realiza la eliminación de prefijos y sufijos, además de realizar una lemmatización.

In [14]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

train['words'] = train['words'].apply(stem_and_lemmatize) #Aplica lematización y Eliminación de Prefijos y Sufijos.
train.head()

Textos_espanol  sdg  \
0  Por ejemplo, el nÃºmero de consultas externas ...    3   
1  En 2007, el gobierno central financió directam...    3   
2  Claramente, hay muchos otros factores en juego...    3   
3  Por ejemplo, el estado australiano de Victoria...    3   
4  El consumo anual de alcohol se estima en 15,7 ...    3   

                                               words  
0  [ejemplo, número, consulta, externa, especiali...  
1  [two thousand and seven, gobierno, cent, finan...  
2  [clara, fact, juego, particul, variedad, barre...  
3  [ejemplo, australiano, victor, creo, agenc, pr...  
4  [consumo, an, alcohol, estim, 157, litro, ano,...

##### **3.4 Selección de campos**

Primero, se separa la variable predictora y los textos que se van a utilizar.

In [15]:
train['words'] = train['words'].apply(lambda x: ' '.join(map(str, x)))
train

Textos_espanol  sdg  \
0     Por ejemplo, el nÃºmero de consultas externas ...    3   
1     En 2007, el gobierno central financió directam...    3   
2     Claramente, hay muchos otros factores en juego...    3   
3     Por ejemplo, el estado australiano de Victoria...    3   
4     El consumo anual de alcohol se estima en 15,7 ...    3   
...                                                 ...  ...   
2995  Un caucus efectivo se basa en fuertes vínculos...    5   
2996  Por el contrario, el porcentaje de hogares en ...    5   
2997  El análisis utilizará una gama de medidas que ...    5   
2998  La capacitación económica, el apoyo y, a veces...    5   
2999  Esto ha sido reconocido por los comitÃ©s en la...    5   

                                                  words  
0     ejemplo número consulta externa especialista t...  
1     two thousand and seven gobierno cent financio ...  
2     clara fact juego particul variedad barrera det...  
3     ejemplo australiano victor creo agenc promoc s...  
4     consumo an alcohol estim 157 litro ano hombr m...  
...                                                 ...  
2995  cauc efectivo bas fuert vinculo grupo nac muj ...  
2996  contrario porcentas hog edad trabaj hombr trab...  
2997  analis utilizar gam medida refles barrera basi...  
2998  capacitac economic apoyo vec incentivo abord j...  
2999  sido reconocido comita atención prestad salud ...  

[3000 rows x 3 columns]

In [16]:
X_datab, y_datab = train['words'],train['sdg']
y_datab

0       3
1       3
2       3
3       3
4       3
       ..
2995    5
2996    5
2997    5
2998    5
2999    5
Name: sdg, Length: 3000, dtype: int64

In [17]:
X_datab

0       ejemplo número consulta externa especialista t...
1       two thousand and seven gobierno cent financio ...
2       clara fact juego particul variedad barrera det...
3       ejemplo australiano victor creo agenc promoc s...
4       consumo an alcohol estim 157 litro ano hombr m...
                              ...                        
2995    cauc efectivo bas fuert vinculo grupo nac muj ...
2996    contrario porcentas hog edad trabaj hombr trab...
2997    analis utilizar gam medida refles barrera basi...
2998    capacitac economic apoyo vec incentivo abord j...
2999    sido reconocido comita atención prestad salud ...
Name: words, Length: 3000, dtype: object

In [18]:
count = CountVectorizer()
X_countb = count.fit_transform(X_datab)

print(X_countb.shape)
print(X_countb.toarray()[0])

(3000, 24511)
[0 0 0 ... 0 0 0]


In [19]:
type(X_countb)

scipy.sparse._csr.csr_matrix

In [20]:
X_countb.shape

(3000, 24511)

In [21]:
count = CountVectorizer(max_features=3000)  # Limita el vocabulario a las 1000 palabras más frecuentes
X_countb = count.fit_transform(X_datab)

In [22]:
X_countb

<3000x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 198058 stored elements in Compressed Sparse Row format>

In [23]:
vocabulario = count.get_feature_names_out()
vocabulario

array(['12', '13', '16', ..., 'zero', 'zona', 'zonas'], dtype=object)

In [24]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=2000)


In [25]:
X_trainb, X_testb, y_trainb, y_testb = train_test_split(X_countb, y_datab, test_size=0.2, random_state=42)


In [26]:
model.fit(X_trainb, y_trainb)


LogisticRegression(max_iter=2000, multi_class='multinomial')

In [27]:
accuracy = model.score(X_testb, y_testb)
print("Exactitud del modelo:", accuracy)


Exactitud del modelo: 0.975


# Creacion Modelo

In [81]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class TextPreprocessor2(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        p = engine()
        preprocessed_data = []
        
        for sentence in X:
            words = sentence.split()
            words = to_lowercase(words)
            words = replace_numbers(words, p)
            words = remove_punctuation(words)
            words = remove_non_ascii(words)
            words = remove_stopwords(words)
            words = remove_specialCoders(words)
            preprocessed_sentence = ' '.join(words)
            preprocessed_data.append(preprocessed_sentence)
        
        return pd.Series(preprocessed_data)

# Crear un pipeline
preprocessing_pipeline = Pipeline([
    ('text_preprocessor', TextPreprocessor2()),
    ('vectorizer',CountVectorizer(max_features=3000))
])

In [82]:
x = train['Textos_espanol']
y = train['sdg']

In [83]:
test = pd.read_excel('SinEtiquetatest_cat_345.xlsx')  # Asegúrate de proporcionar la ruta correcta


In [84]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
x_REAL_test = test["Textos_espanol"]

In [85]:
x_REAL_test

0       Se basa en los nueve años de educación básica ...
1       En la última década, y en particular desde 201...
2       AsÃ, por ejemplo, el estigma asociado a los be...
3       Hay muchos profesores no cualificados en las e...
4       A raíz de su preocupación por el hecho de que ...
                              ...                        
1754    O puede darse de baja llamando a XXXXXXX o vis...
1755    Considere establecer o fortalecer la capacidad...
1756    Sin embargo, la calidad general del entorno de...
1757    Cada vez con más frecuencia, los traficantes d...
1758    También presenta responsabilidad individual, l...
Name: Textos_espanol, Length: 1759, dtype: object

In [86]:
x_train.shape, y_train.shape    

((2250,), (2250,))

In [87]:
x_test.shape, y_test.shape    

((750,), (750,))

In [88]:
preprocessing_pipeline.fit(x_train)

Pipeline(steps=[('text_preprocessor', TextPreprocessor2()),
                ('vectorizer', CountVectorizer(max_features=3000))])

In [89]:
# Aplicar el pipeline a tus datos
preprocessed_train_data = preprocessing_pipeline.transform(x_train)

In [90]:
type(preprocessed_train_data)

scipy.sparse._csr.csr_matrix

In [91]:
preprocessed_train_data

<2250x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 92121 stored elements in Compressed Sparse Row format>

In [92]:
vocabulario = preprocessing_pipeline.named_steps["vectorizer"].get_feature_names_out()
vocabulario

array(['10', '11', '12', ..., 'zero', 'zona', 'zonas'], dtype=object)

# Regresion Logistica Entrenamiento

In [93]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=2000)


In [94]:
X_trainb, X_testb, y_trainb, y_testb = train_test_split(preprocessed_train_data, y_train, test_size=0.2, random_state=42)


In [95]:
model.fit(X_trainb, y_trainb)

LogisticRegression(max_iter=2000, multi_class='multinomial')

In [96]:
accuracy = model.score(X_testb, y_testb)
print("Exactitud del modelo Regresion Logistica:", accuracy)


Exactitud del modelo Regresion Logistica: 0.9688888888888889


In [97]:
X_testb

<450x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 18257 stored elements in Compressed Sparse Row format>

In [98]:
y_testb

2365    5
1283    4
2530    5
1332    4
709     3
       ..
2226    5
1710    4
665     3
2073    5
1107    4
Name: sdg, Length: 450, dtype: int64

In [99]:
y_pred = model.predict(X_testb)

In [100]:
cm = confusion_matrix(y_testb, y_pred)

In [101]:
plt.figure(figsize=(18, 6)) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)

plt.show()

C:\Users\user\AppData\Local\Temp\ipykernel_16776\3281130817.py:5: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [102]:
plt.savefig("confusion_matrix_logist.png")


# Transformacion Pipeline Test

In [103]:
x_REAL_test


0       Se basa en los nueve años de educación básica ...
1       En la última década, y en particular desde 201...
2       AsÃ, por ejemplo, el estigma asociado a los be...
3       Hay muchos profesores no cualificados en las e...
4       A raíz de su preocupación por el hecho de que ...
                              ...                        
1754    O puede darse de baja llamando a XXXXXXX o vis...
1755    Considere establecer o fortalecer la capacidad...
1756    Sin embargo, la calidad general del entorno de...
1757    Cada vez con más frecuencia, los traficantes d...
1758    También presenta responsabilidad individual, l...
Name: Textos_espanol, Length: 1759, dtype: object

In [104]:
preprocessed_REAL_test = preprocessing_pipeline.fit(x_REAL_test)


In [105]:
preprocessed_REAL_test

Pipeline(steps=[('text_preprocessor', TextPreprocessor2()),
                ('vectorizer', CountVectorizer(max_features=3000))])

In [106]:
preprocessed_REAL_test = preprocessing_pipeline.transform(x_REAL_test)


In [145]:
preprocessed_REAL_test

<1759x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 71704 stored elements in Compressed Sparse Row format>

# Predicciones Modelo Test Regresion Logistica

In [146]:
model

LogisticRegression(max_iter=2000, multi_class='multinomial')

In [147]:
predictions = model.predict(preprocessed_REAL_test)


In [148]:
predictions

array([4, 4, 3, ..., 3, 3, 3], dtype=int64)

In [149]:
dataFramePredictedLogit = pd.DataFrame({"Textos_espanol": x_REAL_test, "Predictions": predictions})

# Visualiza el DataFrame
dataFramePredictedLogit

Textos_espanol  Predictions
0     Se basa en los nueve años de educación básica ...            4
1     En la última década, y en particular desde 201...            4
2     AsÃ, por ejemplo, el estigma asociado a los be...            3
3     Hay muchos profesores no cualificados en las e...            3
4     A raíz de su preocupación por el hecho de que ...            3
...                                                 ...          ...
1754  O puede darse de baja llamando a XXXXXXX o vis...            3
1755  Considere establecer o fortalecer la capacidad...            3
1756  Sin embargo, la calidad general del entorno de...            3
1757  Cada vez con más frecuencia, los traficantes d...            3
1758  También presenta responsabilidad individual, l...            3

[1759 rows x 2 columns]

# Naive Bayes Entrenamiento


In [150]:
from sklearn.naive_bayes import MultinomialNB

In [151]:
X_trainb, X_testb, y_trainb, y_testb = train_test_split(preprocessed_train_data, y_train, test_size=0.2, random_state=42)


In [152]:
nb_classifier = MultinomialNB()

In [153]:
nb_classifier.fit(X_trainb, y_trainb)


MultinomialNB()

In [154]:
accuracy = nb_classifier.score(X_testb, y_testb)
print("Exactitud del modelo Naive Bayes:", accuracy)

Exactitud del modelo Naive Bayes: 0.9711111111111111


In [155]:
y_pred = nb_classifier.predict(X_testb)

In [156]:
cm = confusion_matrix(y_testb, y_pred)
plt.figure(figsize=(18, 6)) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)

plt.show()
plt.savefig("confusion_matrix_Bayes.png")


C:\Users\user\AppData\Local\Temp\ipykernel_16776\261665902.py:6: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


# Predicciones Bayes

In [157]:
predictions = nb_classifier.predict(preprocessed_REAL_test)
dataFramePredictedBayes = pd.DataFrame({"Textos_espanol": x_REAL_test, "Predictions": predictions})
dataFramePredictedBayes

Textos_espanol  Predictions
0     Se basa en los nueve años de educación básica ...            3
1     En la última década, y en particular desde 201...            4
2     AsÃ, por ejemplo, el estigma asociado a los be...            3
3     Hay muchos profesores no cualificados en las e...            5
4     A raíz de su preocupación por el hecho de que ...            3
...                                                 ...          ...
1754  O puede darse de baja llamando a XXXXXXX o vis...            3
1755  Considere establecer o fortalecer la capacidad...            3
1756  Sin embargo, la calidad general del entorno de...            3
1757  Cada vez con más frecuencia, los traficantes d...            3
1758  También presenta responsabilidad individual, l...            4

[1759 rows x 2 columns]

# Random Forest Entrenamiento

In [158]:
from sklearn.ensemble import RandomForestClassifier

In [159]:
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)


In [160]:
X_trainb, X_testb, y_trainb, y_testb = train_test_split(preprocessed_train_data, y_train, test_size=0.2, random_state=42)


In [161]:
rf_classifier.fit(X_trainb, y_trainb)

RandomForestClassifier(n_estimators=200, random_state=42)

In [162]:
accuracy = rf_classifier.score(X_testb, y_testb)
print("Exactitud del modelo Random Forest:", accuracy)

Exactitud del modelo Random Forest: 0.9577777777777777


In [163]:
y_pred = rf_classifier.predict(X_testb)

In [164]:
cm = confusion_matrix(y_testb, y_pred)
plt.figure(figsize=(18, 6)) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)

plt.show()
plt.savefig("confusion_matrix_RandomForest.png")


C:\Users\user\AppData\Local\Temp\ipykernel_16776\2362533984.py:6: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


# Predicciones Random Forest

In [165]:
predictions = rf_classifier.predict(preprocessed_REAL_test)
dataFramePredictedForest = pd.DataFrame({"Textos_espanol": x_REAL_test, "Predictions": predictions})
dataFramePredictedForest

Textos_espanol  Predictions
0     Se basa en los nueve años de educación básica ...            4
1     En la última década, y en particular desde 201...            4
2     AsÃ, por ejemplo, el estigma asociado a los be...            3
3     Hay muchos profesores no cualificados en las e...            3
4     A raíz de su preocupación por el hecho de que ...            3
...                                                 ...          ...
1754  O puede darse de baja llamando a XXXXXXX o vis...            3
1755  Considere establecer o fortalecer la capacidad...            3
1756  Sin embargo, la calidad general del entorno de...            3
1757  Cada vez con más frecuencia, los traficantes d...            5
1758  También presenta responsabilidad individual, l...            3

[1759 rows x 2 columns]

# KNN

In [166]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=300)
X_trainb, X_testb, y_trainb, y_testb = train_test_split(preprocessed_train_data, y_train, test_size=0.2, random_state=42)
knn_classifier.fit(X_trainb, y_trainb)
accuracy = knn_classifier.score(X_testb, y_testb)
print("Exactitud del modelo knn_classifier:", accuracy)

Exactitud del modelo knn_classifier: 0.94


# svc

In [167]:
from sklearn.svm import SVC
svm_classifier  = SVC(kernel='linear')
X_trainb, X_testb, y_trainb, y_testb = train_test_split(preprocessed_train_data, y_train, test_size=0.2, random_state=42)
svm_classifier .fit(X_trainb, y_trainb)
accuracy = svm_classifier .score(X_testb, y_testb)
print("Exactitud del modelo svm_classifier :", accuracy)

Exactitud del modelo svm_classifier : 0.96


# GB classifier

In [168]:
from sklearn.ensemble import GradientBoostingClassifier
gb_classifier = GradientBoostingClassifier(n_estimators=200, random_state=42)  # Puedes ajustar el número de estimadores según tus necesidades
X_trainb, X_testb, y_trainb, y_testb = train_test_split(preprocessed_train_data, y_train, test_size=0.2, random_state=42)
gb_classifier .fit(X_trainb, y_trainb)
accuracy = gb_classifier .score(X_testb, y_testb)
print("Exactitud del modelo gb_classifier :", accuracy)

Exactitud del modelo gb_classifier : 0.9444444444444444


# Bayes Bernulli

In [169]:
from sklearn.naive_bayes import BernoulliNB
bnb_classifier = BernoulliNB()
X_trainb, X_testb, y_trainb, y_testb = train_test_split(preprocessed_train_data, y_train, test_size=0.2, random_state=42)
bnb_classifier.fit(X_trainb, y_trainb)
accuracy = bnb_classifier.score(X_testb, y_testb)
print("Exactitud del modelo bnb_classifier :", accuracy)

Exactitud del modelo bnb_classifier : 0.9622222222222222


# Eleccion Modelo Regresion

In [174]:
dataFramePredictedBayes

Textos_espanol  Predictions
0     Se basa en los nueve años de educación básica ...            3
1     En la última década, y en particular desde 201...            4
2     AsÃ, por ejemplo, el estigma asociado a los be...            3
3     Hay muchos profesores no cualificados en las e...            5
4     A raíz de su preocupación por el hecho de que ...            3
...                                                 ...          ...
1754  O puede darse de baja llamando a XXXXXXX o vis...            3
1755  Considere establecer o fortalecer la capacidad...            3
1756  Sin embargo, la calidad general del entorno de...            3
1757  Cada vez con más frecuencia, los traficantes d...            3
1758  También presenta responsabilidad individual, l...            4

[1759 rows x 2 columns]

In [175]:
nombre_archivo = 'PrediccionesBayes.csv'  # Nombre del archivo CSV
dataFramePredictedBayes.to_csv(nombre_archivo, index=False)